In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn.apionly as sns
import matplotlib.pyplot as plt

#Seteo el ano sobre el que voy a medir
ano=2017

/home/dgasch/maestria/lib/python3.5/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


# Analisis sobre Alumnos

## Variables que se decidieron sacar el 20/09
- Pais
- Localidad
- Provincia
- Estado Civil

### Variables que se van a generar
- Se van a generar las variables de EsTecnico como una categorica que es si en el caso de que sea tecnico y no en otro caso, se deben respetar los nulos

In [2]:
#Importo el set de datos de alumnos
alumnos=pd.read_csv(r'/home/dgasch/proyectos/GIAR/alumnos_desertores.csv',sep=';')

#Genero la nueva columna de esTecnico
alumnos['EsTecnico']=alumnos.apply(lambda row: 1 if row['Estudios Secundarios']=='Técnico'  else (row['Estudios Secundarios'] if pd.isnull(row['Estudios Secundarios']) else 0), axis=1)



#### Chequeo que la cantidad sea la misma para las dos columnas

In [3]:
print("Estudios Secundarios:{}".format(alumnos['Estudios Secundarios'].isnull().sum()))
print("EsTecnico: {}".format(alumnos['EsTecnico'].isnull().sum()))
print("Cantidad de registros que estan en el primero y no en el segundo: {}".format(alumnos[(alumnos['EsTecnico'].isnull()) & (alumnos['Estudios Secundarios'].notnull())].shape[0]))

Estudios Secundarios:1089
EsTecnico: 1089
Cantidad de registros que estan en el primero y no en el segundo: 0


In [4]:
#Genero el dataset sin las variables que se decidieron
alumnos_df=alumnos[['Codigo Alumno','EsTecnico','deserto']]

# Analisis sobre cursadas

# Variables que vamos a sacar
- Curso
- Materia
- Departamento
- Ciclo Lectivo ( la vamos a sacar pero sera utilizada para filtrar los registros)
- Modalidad

## Variables que generamos
- edad al ingreso (Ano ingreso - ano nacimiento) 
- Turno una columna por turno con la cantidad 
- Tipo de aprobación por cada una de las variables la cantidad 
- Maximo recursada por materias(existe mas de un registro por alumno y materia)
- Cantidad de veces recursada regular OK
- Descripción de recursada regular OK
- Edad al ingreso
- Edad en ano de ultima actividad

Tener en cuenta que se debe guardar el ciclo lectivo mas actual

In [5]:
#Importo el set de datos de Cursadas
cursadas=pd.read_excel(r'/home/dgasch/proyectos/GIAR/Cursadas-Alumnos-SIGA.xlsx')

#Filtro el set de datos por el ano en que lo quiero medir
cursadas=cursadas.loc[cursadas['Ciclo Lectivo de Cursada']<ano-2,:]

In [6]:
# Genero la variable edad al ingreso
cursadas['edad al ingreso']=cursadas['Año de ingreso']-cursadas['Año de nacimiento']

#Genero la lista de las descripciones de recursadas
recursadas_df=cursadas.groupby(['Cantidad de veces recursada regular','Descripción de recursada regular'])['Materia'].max().reset_index()[['Cantidad de veces recursada regular','Descripción de recursada regular']]

#Agrupo las cursadas de cada alumno por materia y calculo la mayor cantidad de recursadas y lo uno con la descripcion de cada una
cursadas_df=cursadas.groupby(['Codigo Alumno','Materia'])['Cantidad de veces recursada regular'].max().reset_index().join(recursadas_df.set_index('Cantidad de veces recursada regular'),on='Cantidad de veces recursada regular',rsuffix='_o')

#Compruebo con un alumno si se obtuvieron los registros correctos
print(cursadas_df[cursadas_df['Codigo Alumno']==9996270])

#Setea la descripcion como categoria
cursadas_df['Descripción de recursada regular']=cursadas_df['Descripción de recursada regular'].astype('category')

# Elimino la columna materia, y agrupo por alumno y sumarizo las descripciones
recursadas_df=pd.get_dummies(cursadas_df,columns=['Descripción de recursada regular']).drop(['Materia'], axis=1).groupby(['Codigo Alumno']).sum()

       Codigo Alumno                           Materia  \
70723        9996270  Algoritmos y Estructura de Datos   
70724        9996270             Análisis Matemático I   
70725        9996270      Arquitectura de Computadores   
70726        9996270               Matemática Discreta   
70727        9996270                           Química   
70728        9996270         Sistemas y Organizaciones   
70729        9996270     Álgebra y Geometría Analítica   

       Cantidad de veces recursada regular Descripción de recursada regular  
70723                                    1                    Recurso 1 Vez  
70724                                    1                    Recurso 1 Vez  
70725                                    0                       No Recurso  
70726                                    1                    Recurso 1 Vez  
70727                                    0                       No Recurso  
70728                                    0                       No

In [8]:
# Genero las dummies que se acordaron y los valores restantes
cursadas_df=pd.get_dummies(cursadas,columns=['Turno','Tipo de aprobación']).groupby(['Codigo Alumno']).agg({'Ciclo Lectivo de Cursada':['max'],
        'edad al ingreso':['max'], 'Turno_Mañana':['sum'],
       'Turno_Noche':['sum'], 'Turno_Tarde':['sum'], 'Tipo de aprobación_Cambio Curso':['sum'],
        'Tipo de aprobación_Firmo':['sum'],
       'Tipo de aprobación_Libre':['sum'], 'Tipo de aprobación_No Firmo':['sum'],
       'Tipo de aprobación_Promociono':['sum']
        ,'Sexo':['max']})

#Corrijo los nombres de las columnas
cursadas_df.columns = [col[0] for col in cursadas_df.columns.values]

#Uno este DataFrame con el calculado con las descripciones de recursadas y lo guardo en el dataframe de cursadas
cursadas_df=cursadas_df.reset_index().join(recursadas_df,on='Codigo Alumno')

# Finales 

## Columnas a sacar
- Materia
- Ano (Solo se usa para filtrar)

## Variables a generar
- Promedio sobre los maximos de Nota
- Promedio sobre la nota (con aplazos)
- Cantidad de veces que aprobo
- Cantidad de veces que no aprobo
- Cantidad de veces que promociono

In [9]:
#Importo el set de datos de Finales
finales=pd.read_excel(r'/home/dgasch/proyectos/GIAR/Finales-Alumnos-SIGA.xlsx')

#Filtro el set de datos por el ano en que lo quiero medir
finales=finales.loc[finales['Año']<ano-2,:]

In [10]:
#Corrijo los finales para que no haya notas con 11
finales.loc[finales['Nota']==11,['Nota']]=10
#Genero la columna de no aprobado
finales['noAprobado']=finales['Aprobado'].apply(lambda row: 1 if row==0 else 0)
#Agrupo los finales y me quedo con el promedio de las notas maximas
finales_max_df=finales.groupby(['Codigo Alumno','Materia'])['Nota'].max().reset_index().groupby(['Codigo Alumno'])['Nota'].mean().reset_index()

#Seteo el nombre a las columnas para que no se me superpongan
finales_max_df.columns=['Codigo Alumno','Nota_max_prom']

#Genero el resto de las agrupaciones sobre el set de finales
finales_df=finales.groupby(['Codigo Alumno']).agg({'noAprobado':[sum],'Aprobado':[sum],'Promociono':[sum],'Nota':['mean']})

#Corrijo los nombres de las columnas
finales_df.columns = [col[0] for col in finales_df.columns.values]

#Le agrego a los finales la columna de finales maximos agrupados
finales_df=finales_df.reset_index().join(finales_max_df.set_index('Codigo Alumno'),on='Codigo Alumno')

# Realizo la agrupacion de todos los registros

In [26]:
merged_df = alumnos_df.join(cursadas_df.set_index('Codigo Alumno'),on='Codigo Alumno').join(finales_df.set_index('Codigo Alumno'),on='Codigo Alumno')
merged_df=merged_df[(merged_df['Codigo Alumno'].isin(finales_df['Codigo Alumno'])) & (merged_df['Codigo Alumno'].isin(cursadas_df['Codigo Alumno']))]
#Guardo los registros en un csv
merged_df.to_csv(r'/home/dgasch/proyectos/GIAR/baseline_2009.csv',index=False)

In [13]:
merged_df[(merged_df['Codigo Alumno'].isin(finales_df['Codigo Alumno'])) | (merged_df['Codigo Alumno'].isin(cursadas_df['Codigo Alumno']))]

,Codigo Alumno,EsTecnico,deserto
0,10000007,NaN,1
1,10000010,NaN,0
2,10000015,NaN,1
3,10000016,NaN,1
4,10000020,NaN,1
5,10000027,NaN,0
6,10000032,NaN,1
7,10000039,NaN,0
8,10000045,NaN,1
9,10000046,NaN,0


In [33]:
merged_df['deserto'].value_counts()

0    2558
1    2000
Name: deserto, dtype: int64

In [30]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4558 entries, 0 to 8334
Data columns (total 27 columns):
Codigo Alumno                                            4558 non-null int64
EsTecnico                                                3951 non-null float64
deserto                                                  4558 non-null int64
Tipo de aprobación_Libre                                 4558 non-null int64
Sexo                                                     4558 non-null object
Turno_Tarde                                              4558 non-null int64
Tipo de aprobación_Cambio Curso                          4558 non-null int64
Tipo de aprobación_Promociono                            4558 non-null int64
Turno_Noche                                              4558 non-null int64
edad al ingreso                                          4558 non-null int64
Tipo de aprobación_No Firmo                              4558 non-null int64
Turno_Mañana                               

In [19]:
list(merged_df.columns)

['Codigo Alumno',
 'EsTecnico',
 'deserto',
 'Tipo de aprobación_Libre',
 'Sexo',
 'Turno_Tarde',
 'Tipo de aprobación_Cambio Curso',
 'Tipo de aprobación_Promociono',
 'Turno_Noche',
 'edad al ingreso',
 'Tipo de aprobación_No Firmo',
 'Turno_Mañana',
 'Ciclo Lectivo de Cursada',
 'Tipo de aprobación_Firmo',
 'Cantidad de veces recursada regular',
 'Descripción de recursada regular_No Recurso',
 'Descripción de recursada regular_Recurso 1 Vez',
 'Descripción de recursada regular_Recurso 2 Veces',
 'Descripción de recursada regular_Recurso 3 Veces',
 'Descripción de recursada regular_Recurso 4 Veces',
 'Descripción de recursada regular_Recurso 5 Veces',
 'Descripción de recursada regular_Recurso n Veces (>5)',
 'Aprobado',
 'Promociono',
 'noAprobado',
 'Nota',
 'Nota_max_prom']

In [31]:
columnas = ['EsTecnico', 'Tipo de aprobación_Libre',
 'Turno_Tarde',
 'Tipo de aprobación_Cambio Curso',
 'Tipo de aprobación_Promociono',
 'Turno_Noche',
 'edad al ingreso',
 'Tipo de aprobación_No Firmo',
 'Turno_Mañana',
 'Ciclo Lectivo de Cursada',
 'Tipo de aprobación_Firmo',
 'Cantidad de veces recursada regular',
 'Descripción de recursada regular_No Recurso',
 'Descripción de recursada regular_Recurso 1 Vez',
 'Descripción de recursada regular_Recurso 2 Veces',
 'Descripción de recursada regular_Recurso 3 Veces',
 'Descripción de recursada regular_Recurso 4 Veces',
 'Descripción de recursada regular_Recurso 5 Veces',
 'Descripción de recursada regular_Recurso n Veces (>5)',
 'Aprobado',
 'Promociono',
 'noAprobado',
 'Nota']
merged_df[columnas]=merged_df[columnas].astype(int)

    

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
result = pd.concat([df1, df4], axis=1, join='inner')

In [32]:
merged_df.to_csv(r'/home/dgasch/proyectos/GIAR/baseline_2009.csv',index=False)